# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook: Ensemble of Models


### **Mount google drive**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install flair
!pip install transformer

     |████████████████████████████████| 286kB 6.2MB/s 
     |████████████████████████████████| 1.2MB 17.4MB/s 
     |████████████████████████████████| 798kB 28.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 2.1MB 38.6MB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 983kB 42.9MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 901kB 44.7MB/s 
     |████████████████████████████████| 3.3MB 47.4MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=bbca23e1cf6102f7fa5bb458f4244d6b70a544136ac692736f6142b8953eecfb
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Success

### **Load Data and Preprocess**

In [3]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/airlinecomplaint.csv")
print(df.groupby(['label']).size())
df.head()

label
0    1700
1    1700
dtype: int64


,tweet,label
0,@united UA maintenance issues strike again. Fl...,0
1,With @AirCanada taking away more @ACAltitude b...,1
2,@DudePerfect @AmericanAir that's a lot of miss...,1
3,Oh @AmericanAir we promise to try hard not to ...,1
4,@AmericanAir liked to me that I couldn't chang...,0


**Preprocessor to Remove all special characters except emoticons**

In [4]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['tweet'][9])
print(preprocessor(df['tweet'][9]))

@AmericanAir we have been sitting on the plane for two hours at the gate waiting for a pilot. And the power ports aren't working
 americanair we have been sitting on the plane for two hours at the gate waiting for a pilot and the power ports aren't working


In [5]:
df['tweet'] = df['tweet'].apply(preprocessor)

### **Seperate Into Train, Test and Validation Sets**

In [6]:
df_train=df.iloc[0:int(len(df)*0.80)].reset_index(drop=True)
df_test=df.iloc[int(len(df)*0.80):].reset_index(drop=True)

In [7]:
from sklearn.model_selection import train_test_split

df_test, df_eval, sentiment_test, sentiment_eval = train_test_split(df_test['tweet'], df_test['label'], 
                                                                      random_state=1, test_size=.30, 
                                                                      shuffle=True)


print('Length of train set:',len(df_test),'Length of test set:',len(df_eval))

Length of train set: 476 Length of test set: 204


In [8]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def tokenizer(text):
  return [stemmer.stem(word) for word in text.split()]

### **Load All Models and Predict to prepare for Emsemble Model**

****

**Logistic and Naivbayees**

In [9]:
from joblib import load
from sklearn.feature_extraction.text import CountVectorizer

tfidf=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/tfidf_logistic.joblib')
model_reg=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/clf_logistic.joblib')

vec_vocabulary=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/vec_naivebayees.joblib')
vec= CountVectorizer(vocabulary=vec_vocabulary)
model_nb=load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/clf_naivebayees.joblib')

**LSTM Model**

In [10]:
import io
import json
from tensorflow import keras

with open('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/tokenizer.json') as f:
    data = json.load(f)
    tokenizer = keras.preprocessing.text.tokenizer_from_json(data)

model_lstm=keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/model_lstm.h5')

**Flair Model**



In [11]:
from flair.models import TextClassifier

model_flair=TextClassifier.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/resources/taggers/trec/best-model.pt')

2021-05-03 19:34:53,237 loading file /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/resources/taggers/trec/best-model.pt


**Bert Model**

In [13]:
import torch
from transformers import BertForSequenceClassification 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(df.label.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/BERT_ft_epoch7.model',map_location=torch.device('cpu')))

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### **Getting Predictions on Test Data Set**

**NaiveBayees and Logistic Models**

In [14]:
probas=[]
probas_eval=[]

probas.append(model_reg.predict_proba(tfidf.transform(df_test)))
probas.append(model_nb.predict_proba(vec.transform(df_test)))

probas_eval.append(model_reg.predict_proba(tfidf.transform(df_eval)))
probas_eval.append(model_nb.predict_proba(vec.transform(df_eval)))

In [15]:
predictions=[]
predictions.append(model_reg.predict(tfidf.transform(df_test)))
predictions.append(model_nb.predict(vec.transform(df_test)))


predictions_eval=[]
predictions_eval.append(model_reg.predict(tfidf.transform(df_eval)))
predictions_eval.append(model_nb.predict(vec.transform(df_eval)))

**LSTM Model**

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_length= 500

test_sequences = tokenizer.texts_to_sequences(df_test)
test_sequences = pad_sequences(test_sequences,maxlen =max_seq_length)

lstm_pred=model_lstm.predict(test_sequences)
probas.append(lstm_pred)

In [17]:
max_seq_length= 500

eval_sequences = tokenizer.texts_to_sequences(df_eval)
eval_sequences = pad_sequences(eval_sequences,maxlen =max_seq_length)

lstm_pred_eval=model_lstm.predict(eval_sequences)
probas_eval.append(lstm_pred_eval)

In [18]:
preds = np.argmax(lstm_pred, axis=1).flatten()
predictions.append(preds)

preds= np.argmax(lstm_pred_eval, axis=1).flatten()
predictions_eval.append(preds)

**Flair Model**

In [19]:
from flair.data import Sentence

results=[]
for i in df_test.index:
    sentence=Sentence(df_test[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
    else:
      score=sentence.get_labels()[0].score
    results.append([score,1-score])
probas.append(np.array(results))

In [20]:
preds = np.argmax(np.array(results), axis=1).flatten()
predictions.append(preds)

In [21]:
from flair.data import Sentence

results=[]
for i in df_eval.index:
    sentence=Sentence(df_eval[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
    else:
      score=sentence.get_labels()[0].score
    results.append([score,1-score])
probas_eval.append(np.array(results))

In [22]:
preds = np.argmax(np.array(results), axis=1).flatten()
predictions_eval.append(preds)

**Bert Model**

In [23]:
from transformers import BertTokenizer

tokenizerbert = BertTokenizer.from_pretrained(
                  'bert-base-uncased',
                  do_lower_case=True) 


encoded_data_test=tokenizerbert.batch_encode_plus(
                        df_test.values,              # Same we are doing for validation set.
                        add_special_tokens=True,
                        return_attention_mask=True,
                        padding='longest',
                        max_length=256,
                        truncation=True,
                        return_tensors='pt')

encoded_data_eval=tokenizerbert.batch_encode_plus(
                        df_eval.values,              # Same we are doing for validation set.
                        add_special_tokens=True,
                        return_attention_mask=True,
                        padding='longest',
                        max_length=256,
                        truncation=True,
                        return_tensors='pt')

In [24]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

input_ids_test= encoded_data_test['input_ids']
attention_masks_test= encoded_data_test['attention_mask']

dataset_test= TensorDataset(input_ids_test, attention_masks_test,)

dataloader_test = DataLoader(
    dataset_test, 
    sampler=SequentialSampler(dataset_test), 
    batch_size=4
    )

In [25]:
input_ids_eval= encoded_data_eval['input_ids']
attention_masks_eval= encoded_data_eval['attention_mask']

dataset_eval= TensorDataset(input_ids_eval, attention_masks_eval,)

dataloader_eval = DataLoader(
    dataset_eval, 
    sampler=SequentialSampler(dataset_eval), 
    batch_size=4
    )

In [26]:
import torch.nn.functional as F

def predict(dataloader_test):
  
    model.eval()
    all_logits = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        # since we have no loss, the only thing returned is logits
        logits = outputs[0]
        all_logits.append(logits)
    
    all_logits = torch.cat(all_logits, dim=0)
    preds_flat = np.argmax(all_logits.cpu().numpy(), axis=1).flatten()

    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    # get highest prob dimension as prediction
    
    return preds_flat, probs



In [27]:
preds, probs=predict(dataloader_test)
probas.append(probs) 

In [28]:
predictions.append(preds)

In [29]:
preds, probs=predict(dataloader_eval)
probas_eval.append(probs) 

In [30]:
predictions_eval.append(preds)

**Concatenate all Predictions on test data to get one row for each record**

In [31]:
predictions=np.array(predictions)
predictions_eval=np.array(predictions_eval)

In [32]:
probas=np.array(probas)
probas_eval=np.array(probas_eval)

### **Method 1: Using Probabilities and Weighted Majority Voting**

### **Finding Weights of each Model**

Reference: https://machinelearningmastery.com/weighted-average-ensemble-for-deep-learning-neural-networks/

In [33]:
import random
from numpy.linalg import norm

weights = [random.uniform(0, 1)for _ in range(5)]
l1norm = norm(weights,1)
weights= weights / l1norm
print(weights)

[0.24825203 0.13718957 0.16158803 0.18855421 0.26441615]


**Calculate Accuracy with Initial Weights**

In [34]:
from sklearn.metrics import accuracy_score

weightedavg = np.average(probas, axis=0, weights=weights)
result = np.argmax(weightedavg, axis=1)
accuracy_score(result,sentiment_test)

0.819327731092437

**Find Optimal Weights**

In [35]:
from numpy.linalg import norm

def loss_func(weights):
  l1norm = norm(weights,1)
  weights= weights / l1norm
  weightedavg = np.average(probas, axis=0, weights=weights)
  result = np.argmax(weightedavg, axis=1)
  return 1 - accuracy_score(result,sentiment_test)

In [36]:
from scipy.optimize import differential_evolution

bound_w = [(0.0, 1.0)  for _ in range(5)]
result = differential_evolution(loss_func, bound_w, maxiter=1000000, tol=1e-7, disp=True)

differential_evolution step 1: f(x)= 0.165966
differential_evolution step 2: f(x)= 0.165966
differential_evolution step 3: f(x)= 0.161765
differential_evolution step 4: f(x)= 0.161765
differential_evolution step 5: f(x)= 0.161765
differential_evolution step 6: f(x)= 0.161765
differential_evolution step 7: f(x)= 0.161765
differential_evolution step 8: f(x)= 0.161765
differential_evolution step 9: f(x)= 0.161765
differential_evolution step 10: f(x)= 0.161765
differential_evolution step 11: f(x)= 0.161765
differential_evolution step 12: f(x)= 0.159664
differential_evolution step 13: f(x)= 0.157563
differential_evolution step 14: f(x)= 0.157563
differential_evolution step 15: f(x)= 0.157563
differential_evolution step 16: f(x)= 0.157563
differential_evolution step 17: f(x)= 0.157563
differential_evolution step 18: f(x)= 0.157563
differential_evolution step 19: f(x)= 0.157563
differential_evolution step 20: f(x)= 0.157563
differential_evolution step 21: f(x)= 0.157563
differential_evolution

In [37]:
weights=result['x']
l1norm = norm(weights,1)
final_weights= weights / l1norm
print(final_weights)

[0.34757591 0.03956229 0.05851651 0.32466538 0.22967991]


In [38]:
weightedavg = np.average(probas, axis=0, weights=weights)
result = np.argmax(weightedavg, axis=1)
print(accuracy_score(result,sentiment_test))

0.842436974789916


### **Test on Evaluation Set**

In [39]:
weightedavg_eval = np.average(probas_eval, axis=0, weights=weights)
result_eval = np.argmax(weightedavg_eval, axis=1)
print(accuracy_score(result_eval,sentiment_eval))

0.8676470588235294


### **Method 2: Majority Voting using direct predictions**

In [40]:
predictions_t=predictions.T
predictions_eval_t=predictions_eval.T

In [41]:
final_preds=np.array([np.argmax(np.bincount(predictions_t[i],weights=[1,1,1,1,1])) for i in range(predictions_t.shape[0])])
accuracy_score(final_preds,sentiment_test)

0.792016806722689

In [42]:
final_preds_eval=np.array([np.argmax(np.bincount(predictions_eval_t[i],weights=[1,1,1,1,1])) for i in range(predictions_eval_t.shape[0])])
accuracy_score(final_preds_eval,sentiment_eval)

0.8382352941176471